In [6]:
import autoray
from importlib import reload

def tf_qr(x):
    U, s, VH = autoray.do('linalg.svd', x)
    
    dtype = autoray.get_dtype_name(U)
    if 'complex' in dtype:
        s = autoray.astype(s, dtype)
    
    Q = U
    R = autoray.reshape(s, (-1, 1)) * VH
    
    return Q, R


autoray.register_function('tensorflow', 'linalg.qr', tf_qr)

In [3]:
import qubit_networks as beeky
import quimb as qu
import quimb.tensor as qtn
from quimb.tensor.optimize import TNOptimizer

LX, LY = 2, 3
DTYPE = 'complex128'

autodiff_backend = 'tensorflow'
autodiff_backend_opts = {'experimental_compile': True}

def state_energy(
    psi: beeky.QubitEncodeVector,
    hterms: dict,
    vterms: dict,
    **opts):
    '''Energy <psi|H|psi>, summing contribns from
    horiz/vertical terms in Hamiltonian.
    '''

    # TODO: compute row/col envs first?
    he = psi.compute_local_expectation(
        hterms, normalized=True, **opts)

    ve = psi.compute_local_expectation(
        vterms, normalized=True, **opts)        
    
    return autoray.do('real', (he + ve))


In [13]:
import exact_jwt
reload(exact_jwt)

analytical = exact_jwt.HubbardSpinless(LX, LY)
analytical.build_spinless_ham()
Hexact = analytical._Ham.copy()
gex, gsx = qu.eigh(Hexact)
gs_exact = gex[0]
print(f'(Lx={LX}, Ly={LY}) -- GSE EXACT: {gs_exact}')


(Lx=2, Ly=3) -- GSE EXACT: -3.8284271247461894


In [18]:
reload(beeky)
HubSimHam = beeky.SpinlessSimHam(Lx=LX, Ly=LY)

horizontal_terms = dict(HubSimHam.gen_horizontal_ham_terms())
vertical_terms = dict(HubSimHam.gen_vertical_ham_terms())

Hstab = beeky.HamStab(Lx=LX, Ly=LY)
stab_terms = dict(Hstab.gen_ham_terms())

compute_expec_opts = dict(
                cutoff=0.0, 
                max_bond=9, 
                contract_optimize='random-greedy')

psi0 = beeky.QubitEncodeVector.rand(Lx=LX, Ly=LY, bond_dim=2, dtype=DTYPE)

psi0.setup_bmps_contraction_()

    
optmzr = TNOptimizer(
    psi0, # initial state guess
    loss_fn=state_energy,
    constant_tags=['AUX',],
    loss_constants={'hterms': horizontal_terms,
                    'vterms': vertical_terms},
    loss_kwargs=compute_expec_opts,
    autodiff_backend=autodiff_backend,
    **autodiff_backend_opts)


In [21]:
E0 = state_energy(psi0, horizontal_terms, vertical_terms)
print(f'INITIAL: {E0}')

expec_stab_0 = psi0.compute_ham_expec(Hstab)
print(f'STAB0: {expec_stab_0}')

INITIAL: -0.08195621459431111
STAB0: (-0.04546644948219246+1.2303027689128064e-17j)


In [30]:
tn_opt = optmzr.optimize(70)

0%|          | 0/70 [00:45<?, ?it/s]


In [32]:
# tn_opt.compute_ham_expec(Hstab)
state_energy(tn_opt, horizontal_terms, vertical_terms)

-1.199527949434156